In [1]:
import os

In [2]:
%pwd

'/Users/shreyasarkar/name-it/research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'/Users/shreyasarkar/name-it'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from nameit.constants import *
from nameit.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from nameit.logging import logger
from nameit.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
import os
import zipfile


class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        # Add file download implementation here
        pass

    def extract_zip_file(self):
        local_file_path = self.config.local_data_file
        unzip_path = self.config.unzip_dir

        # Ensure the file exists
        if not os.path.exists(local_file_path):
            raise FileNotFoundError(f"File not found: {local_file_path}")

        # Ensure the file is a valid zip archive
        if not zipfile.is_zipfile(local_file_path):
            raise zipfile.BadZipFile(f"File is not a valid zip file: {local_file_path}")

        # Create output directory if it doesn't exist
        os.makedirs(unzip_path, exist_ok=True)

        # Extract the zip file
        try:
            with zipfile.ZipFile(local_file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                print(f"Extraction successful. Files have been extracted to: {unzip_path}")
        except zipfile.BadZipFile as e:
            raise zipfile.BadZipFile(f"Error extracting ZIP file: {e}")
        except Exception as e:
            raise Exception(f"An unexpected error occurred during extraction: {e}")
